In [ ]:
# Dependencies to Visualize the model
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

In [ ]:
# Filepaths, numpy, and Tensorflow
import os
import numpy as np
import tensorflow as tf

In [36]:
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Keras Specific
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [ ]:
import pandas as pd

url = "https://dataclass-project4.s3.amazonaws.com/fer2013.csv"
df = pd.read_csv(url)

In [ ]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [ ]:
# Turning the pixels column into a list of arrays, so each image is an array
X_values = df['pixels'].tolist()
X = []
for i in X_values:
  X_list = [int(x) for x in i.split(' ')]
  X_list = np.asarray(X_list)
  X.append(X_list)

In [ ]:
X = np.asarray(X)
X

array([[ 70,  80,  82, ..., 106, 109,  82],
       [151, 150, 147, ..., 193, 183, 184],
       [231, 212, 156, ...,  88, 110, 152],
       ...,
       [ 17,  17,  16, ..., 154, 133, 113],
       [ 30,  28,  28, ...,  35,  30,  28],
       [ 19,  13,  14, ..., 189, 199, 201]])

In [ ]:
y = pd.get_dummies(df['emotion'])
y.head()

,0,1,2,3,4,5,6
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1


In [37]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [39]:
# Scaling the data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

array([[0.14901961, 0.43921569, 0.7372549 , ..., 0.01568627, 0.01960784,
        0.01960784],
       [0.01176471, 0.00392157, 0.03137255, ..., 0.14901961, 0.18823529,
        0.10980392],
       [0.21960784, 0.15294118, 0.16078431, ..., 0.23529412, 0.21960784,
        0.14509804],
       ...,
       [0.23529412, 0.23921569, 0.25098039, ..., 0.23921569, 0.22745098,
        0.25490196],
       [0.43137255, 0.49019608, 0.52156863, ..., 0.59215686, 0.61960784,
        0.65490196],
       [0.10588235, 0.11764706, 0.12941176, ..., 0.96470588, 0.29803922,
        0.        ]])

In [ ]:
# Creating the model
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=2304))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=7, activation='softmax'))


In [ ]:
# Compiling the model
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
# Fitting the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
# Evaluating the model
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Saving the model
model.save("emotion_model.h5")

In [ ]:
# Loading the model
from tensorflow.keras.models import load_model
emotion_model = load_model('emotion_model.h5')

In [ ]:
# Making predictions
predicted = emotion_model.predict(X_test_scaled)
predicted = np.argmax(predicted, axis=1)
predicted

In [ ]:
# Creating a dataframe of the predictions
predicted_df = pd.DataFrame(predicted)
predicted_df = predicted_df.rename(columns={0: "Predicted"})
predicted_df.head()

In [ ]:
# Creating a dataframe of the actual values
y_test_df = pd.DataFrame(y_test)
y_test_df = y_test_df.rename(columns={0: "Actual"})
y_test_df.head()

In [ ]:
# Merging the two dataframes
merged_df = pd.merge(y_test_df, predicted_df, left_index=True, right_index=True)
merged_df.head()

In [ ]:
# Creating a dataframe of the emotions
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
emotions_df = pd.DataFrame(emotions)
emotions_df = emotions_df.rename(columns={0: "Emotions"})
emotions_df.head()

In [ ]:
# Merging the emotions dataframe with the merged dataframe
merged_df = pd.merge(merged_df, emotions_df, left_on='Actual', right_index=True)
merged_df.head()

In [ ]:
# Merging the emotions dataframe with the merged dataframe
merged_df = pd.merge(merged_df, emotions_df, left_on='Predicted', right_index=True)
merged_df.head()